In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import copy
import rdkit
from tqdm import tqdm
import matplotlib.pyplot as plt
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
import seaborn as sns
from rdkit.Chem import AllChem
from rdkit.Chem.Descriptors import MolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.Lipinski import NumHAcceptors, NumHDonors
from rdkit import DataStructs
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcTPSA
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
from rdkit.Chem.Lipinski import *
from rdkit.Chem.AtomPairs import Torsions, Pairs
from rdkit.Chem import MACCSkeys 
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import ChemicalFeatures
from rdkit.Chem.Pharm2D.SigFactory import SigFactory
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D, Generate
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys 
IPythonConsole.ipython_useSVG=True
from multiprocessing import Pool
from tqdm import tqdm

df = pd.read_csv('./kinase_JAK.csv')
df.head()

,SMILES,measurement_type,measurement_value,Kinase_name
0,C#CCCOC(=O)N1CCC(n2cc(C(N)=O)c(Nc3ccc(F)cc3)n2...,pIC50,6.81,JAK2
1,C#CCCOC(=O)N1CCC(n2cc(C(N)=O)c(Nc3ccc(F)cc3)n2...,pIC50,8.05,JAK1
2,C#CCN(c1ccc(C#N)cn1)C1CCN(c2ncnc3[nH]ccc23)C1,pIC50,10.26,JAK2
3,C#CCN(c1ccc(C#N)cn1)C1CCN(c2ncnc3[nH]ccc23)C1,pIC50,10.26,JAK1
4,C#CCNCC1CCC(c2nnn3cnc4[nH]ccc4c23)CC1,pIC50,7.36,JAK2


rdkit is the most popular cheminformatics package for python. Unfortunately it has awful documentation: http://rdkit.org/docs/index.html

# Train test val splitting

In [ ]:
class DataSplitter:
    def __init__(self, split_ratio):
        self.split_ratio = split_ratio
        
    def split_list(self, input_list):
        # Calculate the length of the split
        split_length = int(len(input_list) * self.split_ratio)

        # Randomly select elements from the input list to create the two output lists
        first_split = np.random.choice(input_list, size=split_length, replace=False)
        second_split = [x for x in input_list if x not in first_split]

        return first_split, second_split
    
    def split_df(self, df, column, test_ratio):
        unique_values = np.unique(df[column])
        first_split, second_split = self.split_list(unique_values)
        X_train = df[df[column].isin(first_split)]
        X_val = df[df[column].isin(second_split)]
        X_val, X_test, y_val, y_test = train_test_split(X_val, X_val, test_size=test_ratio, stratify=X_val['measurement_type']+X_val['Kinase_name'], random_state=42)
        return X_train, X_val, X_test

data_splitter = DataSplitter(0.8)
X_train, X_val, X_test = data

# X_train.to_csv('train.csv',index=False)
# X_val.to_csv('val.csv',index=False)
# X_test.to_csv('test.csv',index=False)

# molecule fingerprints for each smile string

In [ ]:
class FingerprintExtractor:
    def __init__(self):
        self.rf_dict = {}

    def fit(self, smiles_list):
        mol = [Chem.MolFromSmiles(x) for x in smiles_list]

        bit_MFP = np.array([AllChem.GetMorganFingerprintAsBitVect(x,radius = 3, nBits=3*1024) for x in tqdm(mol)], dtype= np.int8)
        bit_frame = pd.DataFrame(bit_MFP)

        rdkbi = {}
        rdfp = np.array([Chem.RDKFingerprint(x, maxPath = 5, bitInfo=rdkbi)for x in tqdm(mol)], dtype = np.int8)
        rdfp = pd.DataFrame(rdfp)

        Mac = [MACCSkeys.GenMACCSKeys(x) for x in tqdm(mol)]
        Mac = np.array(Mac, dtype=np.int8)
        Mac_frame_woHIV= pd.DataFrame(Mac)  

        Mac_Morg_rd_frame = pd.concat([bit_frame, Mac_frame_woHIV, rdfp], axis=1)

        fingerprint = Mac_Morg_rd_frame
        corr = np.corrcoef(np.transpose(fingerprint))
        corr = pd.DataFrame(corr)
        mask = np.triu(np.ones_like(corr, dtype=bool))
        tri_df = corr.mask(mask)

        to_drop = [x for x in tri_df.columns if any(abs(tri_df[x])>0.9)]
        reduced_fingerprint = fingerprint.drop(fingerprint.columns[to_drop], axis = 1)

        for i,j in zip(smiles_list,reduced_fingerprint.values):
            self.rf_dict[i]=j

    def save(self, file_name):
        np.save(file_name, self.rf_dict)
        
fingerprint_extractor = FingerprintExtractor()
fingerprint_extractor.fit(np.unique(df.SMILES))
fingerprint_extractor.save('rf.npy')

100%|██████████| 4528/4528 [00:05<00:00, 789.09it/s]


# molecule conformers for each smile string

In [2]:
def embed_conformers(
    molecule: Chem.rdchem.Mol,
    max_num_conformers: int,
    random_seed: int,
    prune_rms_thresh: float,
    fallback_to_random: bool,
    *,
    use_random: bool = False,
) -> Chem.rdchem.Mol:
  
  mol = copy.deepcopy(molecule)
  params = AllChem.ETKDGv3()

  params.randomSeed = random_seed
  params.pruneRmsThresh = prune_rms_thresh
  params.numThreads = -1
  params.useRandomCoords = use_random

  conf_ids = AllChem.EmbedMultipleConfs(mol, max_num_conformers, params)

  if not conf_ids:
    if not fallback_to_random or use_random:
      raise ValueError('Cant get conformers')
    return _embed_conformers(
        mol,
        max_num_conformers,
        random_seed,
        prune_rms_thresh,
        fallback_to_random,
        use_random=True)
  return mol

def atom_to_feature_vector(
    atom: rdkit.Chem.rdchem.Atom,
    conformer: np.ndarray
):
  """Converts rdkit atom object to feature list of indices.
  Args:
    atom: rdkit atom object.
    conformer: Generated conformers. Returns -1 values if set to None.
  Returns:
    List containing positions (x, y, z) of each atom from the conformer.
  """
  if conformer:
    pos = conformer.GetAtomPosition(atom.GetIdx())
    return [pos.x, pos.y, pos.z]
  return [np.nan, np.nan, np.nan]

In [3]:

def get_conformer_features(x):

    params = AllChem.ETKDGv3()

    params.randomSeed = 42
    params.pruneRmsThresh = 0.01
    params.numThreads = -1
    params.useRandomCoords = False

    mol = copy.deepcopy(Chem.MolFromSmiles(x))
    mol = Chem.AddHs(mol)
    mol =  embed_conformers(mol,1,42,0.01,False)
    AllChem.AlignMolConformers(mol)
    mol = Chem.RemoveHs(mol)

    atom_features_list = []
    conformer = list(mol.GetConformers())[0]
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom, conformer))
    conformer_features = np.array(atom_features_list, dtype=np.float32)
    
    return conformer_features


In [ ]:
smi=np.unique(df.SMILES)

# Create a pool of worker processes
with Pool(processes=4) as p:
    # Use tqdm to track the progress of the map function
    results = dict(zip(smi,p.map(get_conformer_features, smi)))

np.save('cf.npy',results)